# PingPong Latency Matrix

This shows results of running a pingpong latency test on all nodes in ReFrame partition(s).

The result matrices show latency against MPI rank pairs. The rank is not linked to specific hostnames here but as the test uses one rank per node ranks should generally follow the order of nodes shown by e.g. Slurm's `sinfo`.

The final section shows summmary statistics for all partitions.

- TODO: move code into modules

In [56]:
import sys, statistics
from pprint import pprint
sys.path.extend(('../../reframe', '../../')) # the `modules` package imports `reframe` so need that in sys.modules
import modules
import pandas as pd
import matplotlib as mpl
from IPython.display import display, Markdown, HTML

In [57]:
# set notebook width to 90%
display(HTML("<style>.container { width:90% !important; }</style>"))

In [60]:
# find outputs:
paths = [p for p in modules.utils.find_run_outputs(root='../../output', test='Nxnlatbw') if 'build' not in p]
meta = [modules.utils.parse_path_metadata(p) for p in paths]

# load and plot:
summary = {} # sys:part (str) -> stats (dict)
for ix, path in enumerate(paths):
    syspart = '%s:%s' % (meta[ix]['sysname'], meta[ix]['partition'])
    display(Markdown('# %s' % syspart))
    
    # read latencies:
    latencies = {}
    with open(path) as nxn_f:
        for ln, line in enumerate(nxn_f):
            vals = line.strip().split(',')
            if vals[0] == 'src':
                continue
            if len(vals) != 4:
                print('warning: skipping line %i (%i values) beginning: %r ...' % (ln, len(vals), line[0:50]))
                continue
            rankA, rankB, lat, bw = int(vals[0]), int(vals[1]), float(vals[2]), float(vals[3])
            latencies[rankA, rankB] = lat
    
    # get list of node IDs:
    rankAs = sorted(set(k[0] for k in latencies))
    rankBs = sorted(set(k[1] for k in latencies))
    # TODO: could crosscheck these!
    
    # create a dataframe using a list-of-lists:
    lat_list = []
    for rankA in rankAs: # row
        row = [latencies.get((rankA, rankB)) for rankB in rankBs]
        lat_list.append(row)
    df = pd.DataFrame(lat_list)
    dfv = df.stack() # flatten
    summary[syspart] = {
        'count': len(dfv),
        'min': dfv.min(),
        'max': dfv.max(),
        'mean': dfv.mean(),
        'stdev': dfv.std(),
        'range': dfv.max() - dfv.min(),
    }
    sdf = df.style.background_gradient(cmap=mpl.pyplot.get_cmap('Reds')).set_precision(2)
    display(sdf)

# tabulate summary
display(Markdown('# Summary'))
display(pd.DataFrame(summary).sort_index(axis=1).round(2).transpose()) # sort columns by name
    


# csd3:cclake-ib-gcc9-ompi3-ucx

/home/hpcbras1/miniconda3/envs/hpc-tests/lib/python3.7/site-packages/matplotlib/colors.py:527: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


# csd3:cclake-ib-icc19-impi19-ucx

# csd3:cclake-roce-gcc9-ompi3-ucx

# csd3:cclake-roce-icc19-impi19-ucx

# alaska:roce-gcc9-openmpi4-ucx

# alaska:ib-gcc9-openmpi4-ucx

# Summary

count   min   max  mean  stdev  range
alaska:ib-gcc9-openmpi4-ucx         240.0  0.80  1.02  0.90   0.08   0.22
alaska:roce-gcc9-openmpi4-ucx       240.0  1.35  2.14  1.73   0.34   0.79
csd3:cclake-ib-gcc9-ompi3-ucx      3080.0  0.96  1.16  1.00   0.04   0.20
csd3:cclake-ib-icc19-impi19-ucx    3080.0  1.01  1.12  1.02   0.01   0.11
csd3:cclake-roce-gcc9-ompi3-ucx    3080.0  1.39  1.61  1.44   0.03   0.22
csd3:cclake-roce-icc19-impi19-ucx  3080.0  1.43  1.60  1.47   0.02   0.17